In [1]:
import netCDF4
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from glob import glob

### The below function accepts a directory containing netCDF files from Sentinel 5 and converts all the netCDF files in the passed directory to a single geodataframe

In [2]:
def netCDFToDataframe(directory):
    dfList = []
    currentDirectory = os.chdir(directory)
    netCDFFiles = glob("*.nc")
    for file in netCDFFiles:
        data = netCDF4.Dataset(os.path.join(directory, file))
        print(os.path.join(directory, file))
        product = data.groups['PRODUCT']
        latDF = pd.DataFrame(np.array(product.variables['latitude']).flatten(), columns=['latitude'])
        lonDF = pd.DataFrame(np.array(product.variables['longitude']).flatten(), columns=['longitude'])
        N02TropoDF = pd.DataFrame(np.array(product.variables['nitrogendioxide_tropospheric_column']).flatten(), columns = ['N02'])
        df = latDF.merge(lonDF, left_index=True, right_index=True)
        df = df.merge(N02TropoDF, left_index=True, right_index=True)
        dfList.append(df)
    df = pd.concat(dfList)
    geoDF = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    return geoDF